In [35]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

In [47]:
data = pd.read_csv('./data/airbnb_cleaned.csv')
data.drop(columns=['host_id','id','Unnamed: 0'],inplace=True)
X = data.drop(columns=['price'])
y = data['price']

In [48]:
print(X.head())

   host_is_superhost  host_listings_count  neighbourhood  zipcode  latitude  \
0                  0                 45.0             54     8026  41.40889   
1                  0                 45.0             32     8025  41.40420   
2                  0                  2.0             54     8018  41.40560   
3                  1                  5.0             54     8930  41.41203   
4                  1                  1.0             65     8012  41.40145   

   longitude  property_type  room_type  accommodates  bathrooms  ...  beds  \
0    2.18555              1          0             6        1.0  ...   4.0   
1    2.17306              1          0             8        2.0  ...   6.0   
2    2.19821              1          1             2        1.0  ...   1.0   
3    2.22114              1          0             6        2.0  ...   8.0   
4    2.15645              1          1             2        1.0  ...   1.0   

   minimum_nights  has_availability  availability_30  av

In [49]:
object = StandardScaler()
data_scaled = object.fit_transform(X)
# Convert the scaled array back to a DataFrame
X_scaled_df = pd.DataFrame(data_scaled, columns=X.columns)

In [50]:
print(X_scaled_df.head())

   host_is_superhost  host_listings_count  neighbourhood    zipcode  latitude  \
0          -0.481096             0.471105       1.288959   0.115297  1.126927   
1          -0.481096             0.471105       0.278206   0.100143  0.813443   
2          -0.481096            -0.337218       1.288959  -0.005937  0.907020   
3           2.078589            -0.280823       1.288959  13.814727  1.336808   
4           2.078589            -0.356016       1.794336  -0.096862  0.629630   

   longitude  property_type  room_type  accommodates  bathrooms  ...  \
0   1.003127       -0.36815  -1.022099      1.177069  -0.510605  ...   
1   0.309407       -0.36815  -1.022099      2.081581   1.045062  ...   
2   1.706289       -0.36815   0.899552     -0.631956  -0.510605  ...   
3   2.979869       -0.36815  -1.022099      1.177069   1.045062  ...   
4  -0.613147       -0.36815   0.899552     -0.631956  -0.510605  ...   

       beds  minimum_nights  has_availability  availability_30  \
0  0.912995   

In [51]:
# Assuming df1 and df2 are your two DataFrames
merged_df = pd.merge(X_scaled_df, y, left_index=True, right_index=True)

In [ ]:
print(merged_df)

In [39]:
# Shuffle the dataset randomly
X_scaled_df = X_scaled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the dataset into 5 clients
num_clients = 2
data_splits = np.array_split(X_scaled_df, num_clients)

# Create directory to save split files
output_directory = "../client_data/"
os.makedirs(output_directory, exist_ok=True)

# Save each split into separate CSV files
for i, split_data in enumerate(data_splits):
    split_file_path = os.path.join(output_directory, f"client_{i+1}_data.csv")
    split_data.to_csv(split_file_path, index=False)

    print(f"Client {i+1} data saved to {split_file_path}")

d:\Study\Anaconda\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Client 1 data saved to ../client_data/client_1_data.csv
Client 2 data saved to ../client_data/client_2_data.csv


   host_is_superhost  host_listings_count  neighbourhood    zipcode  latitude  \
0          -0.481096            -0.318419       1.105186  -0.051399 -0.824832   
1          -0.481096             3.835981       1.288959  13.814727  1.275314   
2          -0.481096            -0.337218      -0.916321   0.100143  1.023992   
3          -0.481096            -0.337218      -0.916321  -0.081708  0.745265   
4          -0.481096            -0.356016       1.426789  -0.066554 -1.354213   

   longitude  property_type  room_type  accommodates  bathrooms  ...  \
0  -0.632587       -0.36815   0.899552     -0.631956  -0.510605  ...   
1   3.150938        0.11074  -1.022099      0.272556  -0.510605  ...   
2   0.403273       -0.36815   0.899552     -0.631956  -0.510605  ...   
3   0.750411       -0.36815   0.899552     -0.631956  -0.510605  ...   
4  -1.451832       -0.36815   0.899552     -0.631956  -0.510605  ...   

       beds  minimum_nights  has_availability  availability_30  \
0 -0.700929   